In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Computing Post-Optimality Sensitivities of a Constrained Optimization Problem

Lets consider a problem such that we have an active bound and an active inequality constraint.

\begin{align*}
\min_{\theta_0,\, \theta_1} \quad & f(\theta_0, \theta_1; \mathbf{p}) = (\theta_0 - p_0)^2 + \theta_0 \theta_1 + (\theta_1 + p_1)^2 - p_2 \\
\text{where} \quad \mathbf{p} &= \begin{bmatrix} 3 \\ 4 \\ 3 \end{bmatrix} \in \mathbb{R}^3 \\
\text{bounds:} \quad \theta_0 &\le 6 \\
\text{equality constraints:} \quad \theta_0 + \theta_1 &= 0
\end{align*}

We want to know the sensitivities of the optimization outputs with respect to the optimization inputs.

In this context, consider the outputs of the optimization to be the objective and any other functions of interest, $f$.

The design variables $\theta$ and Lagrange multipliers $\lambda$ are effectively the implicit outputs of the optimization.

The _inputs_ to the optimization process consists of:
- any independent parameters, $\bar{p}$
- the bounding values of any **active** design variables, $\bar{b}_{\theta}$
- the bounding values of any **active** constraints, $\bar{b}o_{g}$

In our case we have:

\begin{align*}
    \bar{p} &= \begin{bmatrix} p_0 \\ p_1 \\ p_2 \end{bmatrix} \\
    \bar{b}_{\theta} &= \begin{bmatrix} \theta_0^{ub} \end{bmatrix} \\
    \bar{b}_g &= \begin{bmatrix} g_0^{eq} \end{bmatrix}
\end{align*}

<!-- If active, we can treat the bound on $\theta_0$ as just another equality constraint.

\begin{align*}
  \bar{\mathcal{G}}(\bar{\theta}, \bar{p}) &= \begin{bmatrix}
                                   \theta_0 + \theta_1 \\
                                   \theta_0 - p_3
                                \end{bmatrix} = \bar 0
\end{align*}

**How will my system design ($\bar{\theta}^*$) respond to changes in my assumptions and system inputs ($\bar{p}$)?** -->

### The Universal Derivatives Equation

The UDE is:

\begin{align*}
  \left[ \frac{\partial \mathcal{R}}{\partial \mathcal{u}} \right] \left[ \frac{d u}{d \mathcal{R}} \right]
  &=
  \left[ I \right]
  =
  \left[ \frac{\partial \mathcal{R}}{\partial \mathcal{u}} \right]^T \left[ \frac{d u}{d \mathcal{R}} \right]^T\\
\end{align*}

Here, the residuals are the primal and dual residuals of the optimization process, given above.

## Applying the UDE to solving post-optimality sensitivities

In our case, the unknowns vector consists of
- the optimization parameters ($\bar{p}$)
- the bounding values of any active design variables ($\bar{b}_{\theta}$)
- the bounding values of any active constraints ($\bar{b}_{g}$)
- the design variables of the optimization ($\bar{\theta}$)
- the Lagrange multipliers associated with the active design variables ($\bar{\lambda}_{\theta}$)
- the Lagrange multipliers associated with the active constraints ($\bar{\lambda}_{g}$)
- the objective value **as well as** any other outputs for which we want the sensitivities ($f$)

The total size of the unknowns vector is $N_p + N_{\theta} + 2N_{\lambda \theta} + 2N_{\lambda g} + N_{f}$

\begin{align*}
  \hat{u} &=
  \begin{bmatrix}
    \bar{p} \\
    \bar{b}_{\theta} \\
    \bar{b}_{g} \\
    \bar{\theta} \\
    \bar{\lambda_{\theta}} \\
    \bar{\lambda_{g}} \\
    \bar{f}
  \end{bmatrix}
\end{align*}

Under the UDE, the corresponding residual equations for these unknowns are
- the implicit form of the parameter values
- the implicit form of the active design variable values
- the implicit form of the active constraint values
- the stationarity condition
- the active design variable residuals
- the active constraint residuals
- the implicit form of the explicit calculations of $f$

\begin{align}
\bar{\mathcal{R}}
&=
\begin{bmatrix}
\bar{\mathcal{R}}_p \\
\bar{\mathcal{R}}_{b \theta} \\
\bar{\mathcal{R}}_{b g} \\
\bar{\mathcal{R}}_{\theta} \\
\bar{\mathcal{R}}_{\lambda \theta} \\
\bar{\mathcal{R}}_{\lambda g} \\
\bar{\mathcal{R}}_{f}
\end{bmatrix}
&=
\begin{bmatrix}
  \bar{p} - \check{p} \\[1.1ex]
  \hline \\
  \bar{b}_{\theta} - \check{b}_{\theta} \\[1.1ex]
  \hline \\
  \bar{b}_{g} - \check{b}_g \\[1.1ex]
  \hline \\
  \bar{r}_{\theta} - \left[ \nabla_{\bar{\theta}} \check{f} (\bar{\theta}, \bar{p}) + \nabla_{\bar{\theta}} \check{g}_{\mathcal{A}} (\bar{\theta}, \bar{p})^T \bar{\lambda}_g + \nabla_{\bar{\theta}} \check{\theta}_{\mathcal{A}} (\bar{\theta}, \bar{p})^T \bar{\lambda}_{\theta} \right] \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda \theta} - \left[ \check{\theta}_{\mathcal{A}} \left( \bar{\theta}, \bar{p} \right) - \bar{b}_{\theta} \right] \\[1.1ex]
  \hline \\
  \bar{r}_{\lambda g} - \left[ \check{g}_{\mathcal{A}} \left( \bar{\theta}, \bar{p} \right) - \bar{b}_g \right] \\[1.1ex]
  \hline \\
  f - \check{f}\left(\bar{\theta}, \bar{p} \right) 
\end{bmatrix}
&= \bar 0
\end{align}

In order to find the total derivatives that we seek ($\frac{d f^*}{d \bar{p}}$ and $\frac{d \bar{\theta}^*}{d \bar{p}}$), we need $\frac{\partial \bar{\mathcal{R}}}{\partial \bar{u}}$.

The optimizer has served as the nonlinear solver in this case which has computed the values in the unknowns vector: $\bar{\theta}$, $\bar{\lambda}$, and $\bar{f}$ such that the residuals are satisfied.

\begin{align}
\frac{\partial \bar{\mathcal{R}}}{\partial \bar{u}}
&=
\begin{bmatrix}
\frac{\partial \bar{\mathcal{R}_p}}{\partial \bar{p}} & 0 & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & \frac{\partial \bar{\mathcal{R}_{\bar{b} \theta}}}{\partial \bar{b}_{\theta}} & 0 & 0 & 0 & 0 & 0 \\[1.1ex]
0 & 0 & \frac{\partial \bar{\mathcal{R}_{\bar{b} g}}}{\partial \bar{b}_{g}} & 0 & 0 & 0 & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{p}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\theta}} & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\lambda_{\theta}}} & \frac{\partial \bar{\mathcal{R}_{\theta}}}{\partial \bar{\lambda_g}} & 0 \\[1.1ex]
0 & \frac{\partial \bar{\mathcal{R}_{\lambda \theta}}}{\partial \bar{b}_g} & 0 & \frac{\partial \bar{\mathcal{R}_{\lambda \theta}}}{\partial \bar{\theta}} & \frac{\partial \bar{\mathcal{R}_{\lambda \theta}}}{\partial \bar{\lambda_g}} & 0 & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{p}} & 0 & \frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{b}_g} & \frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{\theta}} & 0 & \frac{\partial \bar{\mathcal{R}}_{\lambda g}}{\partial \bar{\lambda_g}} & 0 \\[1.1ex]
\frac{\partial \bar{\mathcal{R}_f}}{\partial \bar{p}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_f}}{\partial \bar{\theta}} & 0 & 0 & \frac{\partial \bar{\mathcal{R}_f}}{\partial f}
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 \\[1.1ex]
    \frac{\partial \check{\mathcal{L}}}{\partial \bar{p}} & \frac{\partial \nabla \check{\mathcal{L}}}{\partial \bar{\theta}} & \frac{\partial \check{\mathcal{L}}}{\partial \bar{\lambda}} & 0 \\[1.1ex]
    \frac{\partial \check g}{\partial \bar{p}} & \frac{\partial \check g}{\partial \bar{\theta}} & 0 & 0 \\[1.1ex]
    -\frac{\partial \check f}{\partial \bar{p}} & -\frac{\partial \check f}{\partial \bar{\theta}} & 0 & \left[ I_f \right]
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 \\[1.1ex]
    \frac{\partial \nabla \check{\mathcal{L}}}{\partial \bar{p}} & \nabla^2 \check{\mathcal{L}} & \nabla \check g ^T & 0 \\[1.1ex]
    \frac{\partial \check g}{\partial \bar{p}} & \nabla \check g & 0 & 0 \\[1.1ex]
    -\frac{\partial \check f}{\partial \bar{p}} & -\frac{\partial \check f}{\partial \bar{\theta}} & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align}

This nomenclature can be a bit confusing.

**The _partial_ derivatives of the post-optimality residuals are the _total_ derivatives of the analysis.**

In this case of the stationarity residuals $\mathcal{R}_{\bar{\theta}}$, which already include _total_ derivatives of the analysis for the objective and constraint gradients, second derivatives are required.

The corresponding total derivaties which we need to solve for are:

\begin{align}
\frac{d \bar{u}}{d \bar{\mathcal{R}}}
&=
\begin{bmatrix}
\frac{d \bar{p}}{d \bar{\mathcal{R}_p}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_f}} \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{\mathcal{R}_p}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_f}} \\[1.1ex]
\frac{d \bar{\lambda}}{d \bar{\mathcal{R}_p}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_f}} \\[1.1ex]
\frac{d f}{d \bar{\mathcal{R}_p}} & \frac{d f}{d \bar{\mathcal{R}_{\theta}}} & \frac{d f}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d f}{d \bar{\mathcal{R}_f}}
\end{bmatrix}
&=
\begin{bmatrix}
\frac{d \bar{p}}{d \bar{p}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{p}}{d \bar{f}} \\[1.1ex]
\frac{d \bar{\theta}}{d \bar{p}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\theta}}{d \bar{f}} \\[1.1ex]
\frac{d \bar{\lambda}}{d \bar{p}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\lambda}}{d \bar{f}} \\[1.1ex]
\frac{d f}{d \bar{p}} & \frac{d f}{d \bar{\mathcal{R}_{\theta}}} & \frac{d f}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d f}{d \bar{f}}
\end{bmatrix}
\end{align}

The sensitivities of the objective and the design variable values with respect to the parameters of the optimization are highlighted.

In this case, we can solve them with four linear solves of the forward system, or three solves of the reverse system.

TODO: Need to explain how du/dRf becomes du/df.

The UDE for this case, in forward form, is

\begin{align}
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 \\[1.1ex]
    \frac{\partial \nabla \check{\mathcal{L}}}{\partial \bar{p}} & \nabla^2 \check{\mathcal{L}} & \nabla \check g ^T & 0 \\[1.1ex]
    \frac{\partial \check g}{\partial \bar{p}} & \nabla \check g & 0 & 0 \\[1.1ex]
    -\frac{\partial \check f}{\partial \bar{p}} & -\frac{\partial \check f}{\partial \bar{\theta}} & 0 & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
\frac{d \bar{p}}{d \bar{p}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{p}}{d \bar{f}} \\[1.1ex]
\mathbf{\frac{d \bar{\theta}}{d \bar{p}}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\theta}}{d \bar{f}} \\[1.1ex]
\frac{d \bar{\lambda}}{d \bar{p}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\theta}}} & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d \bar{\lambda}}{d \bar{f}} \\[1.1ex]
\mathbf{\frac{d f}{d \bar{p}}} & \frac{d f}{d \bar{\mathcal{R}_{\theta}}} & \frac{d f}{d \bar{\mathcal{R}_{\lambda}}} & \frac{d f}{d \bar{f}}
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 \\[1.1ex]
    0 & \left[ I_\theta \right] & 0 & 0 \\[1.1ex]
    0 & 0 & \left[ I_\lambda \right] & 0 \\[1.1ex]
    0 & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align}

The sensitivities of the objective and the design variable values with respect to the parameters of the optimization are highlighted.

In this case, we have four parameters and thus four columns for which we need to solve the system.

Alternatively, we have three rows of interest in this system...two for the design variables $\theta_0$ and $\theta_1$ and one for the objective $f$.

Taking the transpose and solving this system using the reverse form would require three linear system solves.

\begin{align}
\begin{bmatrix}
    \left[ I_p \right] & \frac{\partial \nabla\check{\mathcal{L}}}{\partial \bar{p}}^T & \frac{\partial \check g}{\partial \bar{p}}^T & -\frac{\partial \check f}{\partial \bar{p}}^T \\[1.1ex]
    0 & \nabla^2 \check{\mathcal{L}}^T & \nabla \check g & -\frac{\partial \check f}{\partial \bar{\theta}}^T \\[1.1ex]
    0 & \nabla \check g ^T & 0 & 0 \\[1.1ex]
    0 & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\begin{bmatrix}
\frac{d \bar{p}}{d \bar{p}} & \mathbf{\frac{d \bar{\theta}}{d \bar{p}}^T} & \frac{d \bar{\lambda}}{d \bar{p}}^T & \mathbf{\frac{d f}{d \bar{p}}^T} \\[1.1ex]
\frac{d \bar{p}}{d \bar{\mathcal{R}_{\theta}}}^T & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\theta}}}^T & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\theta}}}^T & \frac{d f}{d \bar{\mathcal{R}_{\theta}}}^T \\[1.1ex]
\frac{d \bar{p}}{d \bar{\mathcal{R}_{\lambda}}}^T & \frac{d \bar{\theta}}{d \bar{\mathcal{R}_{\lambda}}}^T & \frac{d \bar{\lambda}}{d \bar{\mathcal{R}_{\lambda}}}^T & \frac{d f}{d \bar{\mathcal{R}_{\lambda}}}^T \\[1.1ex]
\frac{d \bar{p}}{d \bar{f}}^T & \frac{d \bar{\theta}}{d \bar{f}}^T & \frac{d \bar{\lambda}}{d \bar{f}}^T & \frac{d f}{d \bar{f}}^T
\end{bmatrix}
&=
\begin{bmatrix}
    \left[ I_p \right] & 0 & 0 & 0 \\[1.1ex]
    0 & \left[ I_\theta \right] & 0 & 0 \\[1.1ex]
    0 & 0 & \left[ I_\lambda \right] & 0 \\[1.1ex]
    0 & 0 & 0 & \left[ I_f \right]
\end{bmatrix}
\end{align}

## Working through the example

First, lets use OpenMDAO to find the solution.

In [2]:
import jax.numpy as np
import openmdao.api as om


class ObjComp(om.JaxExplicitComponent):

    def setup(self):
        self.add_input('Θ', shape=(2,))
        self.add_input('p', shape=(4,))
        self.add_output('f', shape=(1,))

    def compute_primal(self, Θ, p):
        f = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
        return np.array([f])

class ConComp(om.JaxExplicitComponent):

    def setup(self):
        self.add_input('Θ', shape=(2,))
        self.add_input('p', shape=(4,))
        self.add_output('g', shape=(1,))

    def compute_primal(self, Θ, p):
        g = Θ[0] + Θ[1]
        return np.array([g])


_prob = om.Problem()
_prob.model.add_subsystem('f_comp', ObjComp(), promotes_inputs=['*'], promotes_outputs=['*'])
_prob.model.add_subsystem('g_comp', ConComp(), promotes_inputs=['*'], promotes_outputs=['*'])

_prob.model.add_design_var('Θ', upper=[6., None])
_prob.model.add_constraint('g', equals=0.)
_prob.model.add_objective('f')

_prob.driver = om.ScipyOptimizeDriver()

_prob.setup()

_prob.set_val('p', [3, 4, 3, 6])

_prob.run_driver()


Optimization terminated successfully    (Exit mode 0)
            Current function value: -25.999999999999993
            Iterations: 2
            Function evaluations: 2
            Gradient evaluations: 2
Optimization Complete
-----------------------------------


Problem: problem
Driver:  ScipyOptimizeDriver
  success     : True
  iterations  : 3
  runtime     : 1.6263E-01 s
  model_evals : 3
  model_time  : 5.5082E-02 s
  deriv_evals : 2
  deriv_time  : 1.0327E-01 s
  exit_status : SUCCESS

In [3]:
_prob.model.list_vars(print_arrays=True);

6 Variables(s) in 'model'

varname  val                  io      prom_name
-------  -------------------  ------  ---------
f_comp
  Θ      |8.48528137|         input   Θ        
         val:
         array([ 6., -6.])
  p      |8.36660027|         input   p        
         val:
         array([3., 4., 3., 6.])
  f      [-26.]               output  f        
g_comp
  Θ      |8.48528137|         input   Θ        
         val:
         array([ 6., -6.])
  p      |8.36660027|         input   p        
         val:
         array([3., 4., 3., 6.])
  g      [1.77635684e-15]     output  g        




In [4]:
active_dvs, active_cons = _prob.driver.compute_lagrange_multipliers()

Now lets form the UDE system and compute the sensitivities, outside of OpenMDAO first.

In [5]:
# Convert OpenMDAO values to jax arrays

f_opt = np.array(_prob.get_val('f'))
Θ_opt = np.array(_prob.get_val('Θ'))
p = np.array(_prob.get_val('p'))

# The lagrange multipliers of the active constraints are
λ_opt = np.array([active_dvs['Θ']['multipliers'][active_dvs['Θ']['indices']],
              active_cons['g']['multipliers'][active_cons['g']['indices']]])

Define our objective and active constraint (and bounds) functions.

In [6]:
def f(Θ, p):
    f = (Θ[0] - p[0])**2 + Θ[0] * Θ[1] + (Θ[1] + p[1])**2 - p[2]
    return np.array([f])

def g_active(Θ, p):
    return np.array([Θ[0] + Θ[1],
                     Θ[0] - p[3]])

# def df_dΘ(Θ, p):
#     df_dΘ = [[2 * (Θ[0] - p[0]) + Θ[1]],
#              [Θ[0] + 2 * (Θ[1] + p[1])]]
#     return np.array(df_dΘ)

# def df_dp(Θ, p):
#     df_dp = [[-2 * (Θ[0] - p[0])],
#              [2 * (Θ[1] + p[1])],
#              [-1],
#              [0]]
#     return np.array(df_dp)

In [7]:
import jax

# The design vars, from OpenMDAO
print('\nΘ*:')
print(Θ_opt)

# The Lagrange multipliers, from OpenMDAO
print("\nλ*:")
print(λ_opt)

# Jacobian of f with respect to Θ
df_dΘ = jax.jacobian(f, argnums=0)(Θ_opt, p)
print("\n∂f/∂Θ:")
print(df_dΘ)

# Jacobian of f with respect to p
df_dp = jax.jacobian(f, argnums=1)(Θ_opt, p)
print("\n∂f/∂p:")
print(df_dp)

# Jacobian of g_active with respect to Θ
dg_dΘ = jax.jacobian(g_active, argnums=0)(Θ_opt, p)
print("\n∂g/∂Θ:")
print(dg_dΘ)

# Jacobian of g_active with respect to p
dg_dp = jax.jacobian(g_active, argnums=1)(Θ_opt, p)
print("\n∂g/∂p:")
print(dg_dp)

# Lagrangian
dL_dΘ = -df_dΘ.T + np.matmul(dg_dΘ.T, λ_opt)
print("\n∇L:")
print(dL_dΘ)

# Hessian of the objective
d2f_dΘ2 = jax.jacobian(jax.jacobian(f, argnums=0), argnums=0)(Θ_opt, p)
print("\n∇²f (via jacobian of jacobian):")
print(d2f_dΘ2)

d2f_dΘdp = jax.jacobian(jax.jacobian(f, argnums=0), argnums=1)(Θ_opt, p)
print("\n∂∇f/∂p (via jacobian of jacobian):")
print(d2f_dΘdp)

# Hessian of the constraints
d2g_dΘ2 = jax.jacobian(jax.jacobian(g_active, argnums=0), argnums=0)(Θ_opt, p)
print("\n∇²g (via jacobian of jacobian):")
print(d2g_dΘ2)

d2g_dΘdp = jax.jacobian(jax.jacobian(g_active, argnums=0), argnums=1)(Θ_opt, p)
print("\n∂∇g/∂p (via jacobian of jacobian):")
print(d2g_dΘdp)

# # Hessian of the lagrangian
d2L_dΘ2 = -d2f_dΘ2.T + np.dot(d2g_dΘ2.T, λ_opt)
print("\n∇²L:")
print(d2L_dΘ2)

# # Hessian of the lagrangian
d2L_dΘdp = -d2f_dΘ2.T + np.dot(d2g_dΘ2.T, λ_opt)
print("\n∇²L:")
print(d2L_dΘ2)

I_p = np.eye(4)
print("\nI_p:")
print(I_p)

I_f = np.eye(1)
print("\nI_f:")
print(I_f)


Θ*:
[ 6. -6.]

λ*:
[[ 2.]
 [-2.]]

∂f/∂Θ:
[[1.77635684e-15 2.00000000e+00]]

∂f/∂p:
[[-6. -4. -1.  0.]]

∂g/∂Θ:
[[1. 1.]
 [1. 0.]]

∂g/∂p:
[[ 0.  0.  0. -0.]
 [ 0.  0.  0. -1.]]

∇L:
[[-3.10862447e-15]
 [-1.77635684e-15]]

∇²f (via jacobian of jacobian):
[[[2. 1.]
  [1. 2.]]]

∂∇f/∂p (via jacobian of jacobian):
[[[-2.  0.  0.  0.]
  [ 0.  2.  0.  0.]]]

∇²g (via jacobian of jacobian):
[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]

∂∇g/∂p (via jacobian of jacobian):
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]]

∇²L:
[[[-2.]
  [-1.]]

 [[-1.]
  [-2.]]]

∇²L:
[[[-2.]
  [-1.]]

 [[-1.]
  [-2.]]]

I_p:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

I_f:
[[1.]]


In [8]:
import scipy.sparse as sp

def assemble_ude_matrix_scipy_sparse(nabla2_L, nabla_g, dg_dp, df_dtheta, df_dp, Np, Nx, Ng):
    """
    Assemble the UDE matrix using SciPy sparse matrices.
    This is more memory efficient for large problems.
    """

    # Convert JAX arrays to numpy for SciPy
    nabla2_L_np = np.array(nabla2_L).reshape((Nx, Nx))
    nabla_g_np = np.array(nabla_g)
    dg_dp_np = np.array(dg_dp)
    df_dtheta_np = np.array(df_dtheta)
    df_dp_np = np.array(df_dp)

    # Create sparse blocks

    # Row 1: [I_p, 0, 0, 0]
    row1 = [
        sp.eye(Np),                                    # I_p
        sp.csr_matrix((Np, Nx)),                      # 0
        sp.csr_matrix((Np, Ng)),                      # 0
        sp.csr_matrix((Np, 1))                        # 0
    ]

    # Row 2: [∂∇L/∂p, ∇²L, ∇g^T, 0]
    row2 = [
        sp.csr_matrix((Nx, Np)),                      # ∂∇L/∂p (placeholder)
        sp.csr_matrix(nabla2_L_np),                   # ∇²L
        sp.csr_matrix(nabla_g_np.T),                  # ∇g^T
        sp.csr_matrix((Nx, 1))                        # 0
    ]

    # Row 3: [∂g/∂p, ∇g, 0, 0]
    row3 = [
        sp.csr_matrix(dg_dp_np),                      # ∂g/∂p
        sp.csr_matrix(nabla_g_np),                    # ∇g
        sp.csr_matrix((Ng, Ng)),                      # 0
        sp.csr_matrix((Ng, 1))                        # 0
    ]

    # Row 4: [-∂f/∂p, -∂f/∂θ, 0, I_f]
    row4 = [
        sp.csr_matrix(-df_dp_np),                     # -∂f/∂p
        sp.csr_matrix(-df_dtheta_np),                 # -∂f/∂θ
        sp.csr_matrix((1, Ng)),                       # 0
        sp.eye(1)                                     # I_f
    ]

    # Assemble using bmat
    partial_R_partial_u = sp.bmat([row1, row2, row3, row4], format='csr')

    return partial_R_partial_u

In [9]:
partial_R_partial_u = assemble_ude_matrix_scipy_sparse(d2L_dΘ2, dg_dΘ, dg_dp, df_dΘ, df_dp, Np=4, Nx=2, Ng=2)

In [10]:
with np.printoptions(linewidth=1024, precision=1):
    print(np.asarray(partial_R_partial_u.todense(), dtype=int))

[[ 1  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -2 -1  1  1  0]
 [ 0  0  0  0 -1 -2  1  0  0]
 [ 0  0  0  0  1  1  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0]
 [ 6  3  1  0  0 -2  0  0  1]]


To obtain the sensitivities of the objective with respect to the parameters, the final row of $\frac{d u}{d \mathcal{R}}$, we transpose $\frac{\partial \mathcal{R}}{\partial u}$ and seed the right hand side with a 1 in the last row.

\begin{align}
  \begin{bmatrix}
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\[1.5ex]
    0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\[1.5ex]
    0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\[1.5ex]
    0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\[1.5ex]
    0 & 0 & 0 & 0 &-2 &-1 & 1 & 1 & 0 \\[1.5ex]
    0 & 0 & 0 & 0 &-1 &-2 & 1 & 0 & 0 \\[1.5ex]
    0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 0 \\[1.5ex]
    0 & 0 & 0 &-1 & 1 & 0 & 0 & 0 & 0 \\[1.5ex]
    6 & 3 & 1 & 0 & 0 &-2 & 0 & 0 & 1
  \end{bmatrix}
  ^T
  \begin{bmatrix}
    \frac{d f^*}{d p_0} \\[1.3ex]
    \frac{d f^*}{d p_1} \\[1.3ex]
    \frac{d f^*}{d p_2} \\[1.3ex]
    \frac{d f^*}{d p_3} \\[1.3ex]
    \frac{d f^*}{d r_\theta 0} \\[1.3ex]
    \frac{d f^*}{d r_\theta 1} \\[1.3ex]
    \frac{d f^*}{d r_\lambda 0} \\[1.3ex]
    \frac{d f^*}{d r_\lambda 1} \\[1.3ex]
    \frac{d f^*}{d f^*}
  \end{bmatrix}
  &=
  \begin{bmatrix}
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    0 \\[1.5ex]
    1
  \end{bmatrix}  
\end{align}

In [11]:
from scipy.sparse.linalg import spsolve
rhs = sp.csr_matrix([[0, 0, 0, 0, 0, 0, 0, 0, 1]]).T
dfstar_du = spsolve(partial_R_partial_u.T, rhs)

In [12]:
dfstar_du

array([-6., -4., -1., -2., -0., -0.,  2., -2.,  1.])

In [13]:
dfstar_dp = dfstar_du[:4]

In [14]:
dfstar_dp

array([-6., -4., -1., -2.])

## Checking the results

Recall that the optimal objective value $f^*$ was

In [15]:
f_opt.item()

-25.999999999999993

Lets check the sensitivities by perturbing each element in p and reoptimizing

In [16]:
def check_f_sensitivity(h=1.0E-8):
    _prob.driver.options['disp'] = False

    print(f'Sensitivity               {"UDE Result":20s}       {"FD Result":20s}          {"Error":20s}')

    for p_idx in range(4):
        p_nom = np.array([3, 4, 3, 6])
        ub_nom = np.array([6., 1.0E16])

        dp = np.zeros(4)
        dp = dp.at[p_idx].set(h)

        dub = np.zeros(2)
        if p_idx == 3:
            # To test p3 we need to change the upper bound on θ
            dub = dub.at[0].set(h)

        _prob.set_val('p', p_nom + dp)
        _prob.set_val('Θ', [1, 1]) # Start away from the optimum

        _prob.model.set_design_var_options('Θ', upper=ub_nom + dub)

        _prob.run_driver()
        _prob.set_val('p', p_nom)
        _prob.model.set_design_var_options('Θ', upper=ub_nom)

        dfstar_dpi_fd = (_prob.get_val('f') - f_opt) / h

        print(f'   df*/dp_{p_idx}     {dfstar_dp[p_idx]:20.12f}      {dfstar_dpi_fd[0]:20.12f}      {dfstar_dp[p_idx]-dfstar_dpi_fd[p_idx]:20.12f}')

In [17]:
check_f_sensitivity()

Sensitivity               UDE Result                 FD Result                     Error               
   df*/dp_0          -6.000000000000           -6.000000496442            0.000000496442
   df*/dp_1          -4.000000000000           -3.999999975690           -0.000000024310
   df*/dp_2          -1.000000000000           -1.000000082740            0.000000082740
   df*/dp_3          -2.000000000000           -2.000000876023            0.000000876023


Similarly, for the sensitivities of $\theta^*$

In [18]:
rhs = sp.csr_matrix([[0, 0, 0, 0, 1, 0, 0, 0, 0]]).T
dthetastar0_du = spsolve(partial_R_partial_u.T, rhs)
rhs = sp.csr_matrix([[0, 0, 0, 0, 0, 1, 0, 0, 0]]).T
dthetastar1_du = spsolve(partial_R_partial_u.T, rhs)
dthetastar_du = np.vstack((dthetastar0_du, dthetastar1_du))

In [19]:
dthetastar_dp = dthetastar_du[:, :4]

In [20]:
dthetastar_dp

Array([[ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0., -1.]], dtype=float64)

That is, at the optimum $\theta_0$ is on its upper bound, so modifying this upper bound will necessarily change $\theta_0$ since we assume the bound remains active.

Since $\theta_1$ is constrained to be equal and opposite to $\theta_0$, the increase in $\theta_0$ will result in a equal and opposite change in $\theta_1$.

In [21]:
def check_Θ_sensitivity(h=1.0E-8):
    _prob.driver.options['disp'] = False

    print(f'Sensitivity                                       '
          f'{"UDE Result":20s}                      '
          f'{"FD Result":20s}                         '
          f'{"Error":20s}')

    for p_idx in range(4):
        p_nom = np.array([3, 4, 3, 6])
        ub_nom = np.array([6., 1.0E16])

        dp = np.zeros(4)
        dp = dp.at[p_idx].set(h)

        dub = np.zeros(2)
        if p_idx == 3:
            # To test p3 we need to change the upper bound on θ
            dub = dub.at[0].set(h)

        _prob.set_val('p', p_nom + dp)
        _prob.set_val('Θ', [1, 1]) # Start away from the optimum

        _prob.model.set_design_var_options('Θ', upper=ub_nom + dub)

        _prob.run_driver()
        _prob.set_val('p', p_nom)
        _prob.model.set_design_var_options('Θ', upper=ub_nom)

        dthetastar_dpi_fd = (_prob.get_val('Θ') - Θ_opt) / h

        # print(prob.get_val('Θ'), Θ_opt, dthetastar_dpi_fd)

        with np.printoptions(precision=4, formatter={'all':lambda x: f'{x:16.12f}'}):
            print(f'   dΘ*/dp_{p_idx}              {dthetastar_dp[:, p_idx]}      {dthetastar_dpi_fd}      {dthetastar_dp[:, p_idx]-dthetastar_dpi_fd}')

In [22]:
check_Θ_sensitivity()

Sensitivity                                       UDE Result                                FD Result                                    Error               
   dΘ*/dp_0              [  0.000000000000   0.000000000000]      [  0.000000000000  -0.000000177636]      [  0.000000000000   0.000000177636]
   dΘ*/dp_1              [  0.000000000000   0.000000000000]      [  0.000000000000   0.000000000000]      [  0.000000000000   0.000000000000]
   dΘ*/dp_2              [  0.000000000000   0.000000000000]      [  0.000000000000   0.000000000000]      [  0.000000000000   0.000000000000]
   dΘ*/dp_3              [  1.000000000000  -1.000000000000]      [  0.999999993923  -1.000000171558]      [  0.000000006077   0.000000171558]


## A matrix-free approach

In [80]:
class OptimizationProblem(om.ExplicitComponent):
    """
    Component that defines the optimization problem:
    min f(θ₀, θ₁; p) = (θ₀ - p₀)² + θ₀θ₁ + (θ₁ + p₁)² - p₂
    """

    def setup(self):
        # Inputs
        self.add_input('theta0', val=1.0)
        self.add_input('theta1', val=1.0)
        self.add_input('p0', val=3.0)
        self.add_input('p1', val=4.0)
        self.add_input('p2', val=3.0)

        # Outputs
        self.add_output('f', val=1.0)

        # Declare partials
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        theta0 = inputs['theta0']
        theta1 = inputs['theta1']
        p0 = inputs['p0']
        p1 = inputs['p1']
        p2 = inputs['p2']

        outputs['f'] = (theta0 - p0)**2 + theta0*theta1 + (theta1 + p1)**2 - p2

    def compute_partials(self, inputs, partials):
        theta0 = inputs['theta0']
        theta1 = inputs['theta1']
        p0 = inputs['p0']
        p1 = inputs['p1']

        # Derivatives of f
        partials['f', 'theta0'] = 2*(theta0 - p0) + theta1
        partials['f', 'theta1'] = theta0 + 2*(theta1 + p1)
        partials['f', 'p0'] = -2*(theta0 - p0)
        partials['f', 'p1'] = 2*(theta1 + p1)
        partials['f', 'p2'] = -1.0


class EqualityConstraint(om.ExplicitComponent):
    """
    Equality constraint: θ₁ = -θ₀
    Formulated as g₁ = θ₀ + θ₁ = 0
    """

    def setup(self):
        self.add_input('theta0', val=1.0)
        self.add_input('theta1', val=1.0)
        self.add_output('g1', val=0.0)

        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['g1'] = inputs['theta0'] + inputs['theta1']

    def compute_partials(self, inputs, partials):
        partials['g1', 'theta0'] = 1.0
        partials['g1', 'theta1'] = 1.0


class BoundConstraint(om.ExplicitComponent):
    """
    Upper bound constraint: θ₀ ≤ θ₀_ub
    Formulated as g₂ = θ₀ - θ₀_ub ≤ 0
    When active, treated as g₂ = θ₀ - θ₀_ub = 0
    """

    def setup(self):
        self.add_input('theta0', val=1.0)
        self.add_input('theta0_ub', val=6.0)
        self.add_output('g2', val=0.0)

        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['g2'] = inputs['theta0'] - inputs['theta0_ub']

    def compute_partials(self, inputs, partials):
        partials['g2', 'theta0'] = 1.0
        partials['g2', 'theta0_ub'] = -1.0

def main():
    """
    Main function to demonstrate the UDE sensitivity analysis.
    """
    # Create OpenMDAO problem
    prob = om.Problem()

    # Add subsystems
    prob.model.add_subsystem('obj', OptimizationProblem(),
                            promotes_inputs=['theta0', 'theta1', 'p0', 'p1', 'p2'],
                            promotes_outputs=['f'])

    prob.model.add_subsystem('eq_con', EqualityConstraint(),
                            promotes_inputs=['theta0', 'theta1'],
                            promotes_outputs=['g1'])

    prob.model.add_subsystem('bound_con', BoundConstraint(),
                            promotes_inputs=['theta0', 'theta0_ub'],
                            promotes_outputs=['g2'])

    # Add driver (optimizer)
    prob.driver = om.ScipyOptimizeDriver()
    prob.driver.options['optimizer'] = 'SLSQP'
    prob.driver.options['tol'] = 1e-8
    prob.driver.options['disp'] = False

    # Define design variables
    prob.model.add_design_var('theta0', lower=-50, upper=6.0)
    prob.model.add_design_var('theta1', lower=-50, upper=50)

    # Define objective
    prob.model.add_objective('f')

    # Define constraints
    prob.model.add_constraint('g1', equals=0.0)

    # Setup the problem
    prob.setup()

    # Set parameter values
    prob.set_val('p0', 3.0)
    prob.set_val('p1', 4.0)
    prob.set_val('p2', 3.0)
    prob.set_val('theta0_ub', 6.0)

    # Initial guess
    prob.set_val('theta0', 5.0)
    prob.set_val('theta1', -5.0)

    # Run optimization
    print("\n" + "="*60)
    print("Running Optimization")
    print("="*60)
    prob.run_driver()

    # Print results
    print("\n" + "="*60)
    print("Optimization Results")
    print("="*60)
    print(f"θ₀* = {prob.get_val('theta0')[0]:.6f}")
    print(f"θ₁* = {prob.get_val('theta1')[0]:.6f}")
    print(f"f* = {prob.get_val('f')[0]:.6f}")
    print(f"g₁ = {prob.get_val('g1')[0]:.6e} (should be ~0)")
    print(f"g₂ = {prob.get_val('g2')[0]:.6f}")

    # Check if bound is active
    bound_active = abs(prob.get_val('theta0')[0] - 6.0) < 1e-6
    print(f"Bound active: {'bound_active'}")

    # Determine active constraints
    active_constraints = ['g1']  # Equality always active
    if bound_active:
        active_constraints.append('g2')

    # Compute Lagrange multipliers
    print("\n" + "="*60)
    print("Computing Lagrange Multipliers")
    print("="*60)

    # Get gradients at optimum
    grad_f = prob.compute_totals(of='f', wrt=['theta0', 'theta1'], return_format='array').flatten()
    print(f"∇f = {grad_f}")

    # Build constraint Jacobian
    jac_g = []
    for con in active_constraints:
        grad_g = prob.compute_totals(of=con, wrt=['theta0', 'theta1'], return_format='array').flatten()
        jac_g.append(grad_g)
        print(f"∇{con} = {grad_g}")
    jac_g = np.array(jac_g)

    # Solve for multipliers: ∇f = Σλᵢ∇gᵢ
    lambda_vals = np.linalg.lstsq(jac_g.T, -grad_f, rcond=None)[0]
    print(f"Lagrange multipliers: {lambda_vals}")

    # Verify KKT conditions
    residual = grad_f + jac_g.T @ lambda_vals
    print(f"KKT residual: {residual} (should be ~0)")

    # Create UDE solver
    print("\n" + "="*60)
    print("Computing Post-Optimality Sensitivities")
    print("="*60)

    ude_solver = PostOptimalitySensitivitySolver(
        prob=prob,
        design_vars=['theta0', 'theta1'],
        parameters=['p0', 'p1', 'p2', 'theta0_ub'],
        outputs=['f'],
        verbose=False  # Set to True for debugging
    )

    jac_g_act = ude_solver.compute_active_constraint_jacobian()

    print(jac_g_act.todense())

    jvp = ude_solver.compute_active_constraint_jvp(v_theta=np.array([1.0, 0.0]))

    print(jvp)





    # # # Set Lagrange multipliers
    # # ude_solver.lambda_vals = lambda_vals

    # # Compute sensitivities
    # sensitivities = ude_solver.solve_sensitivities()

    # # Display results
    # print("\n" + "="*60)
    # print("Sensitivity Results")
    # print("="*60)

    # param_names = ['p₀', 'p₁', 'p₂', 'θ₀_ub']

    # print("\nDesign Variable Sensitivities (dθ/dp):")
    # print("-" * 40)
    # dtheta_dp = sensitivities['dtheta_dp']
    # for i, var in enumerate(['θ₀', 'θ₁']):
    #     print(f"{var}:")
    #     for j, param in enumerate(param_names):
    #         print(f"  d{var}/d{param} = {dtheta_dp[i, j]:+.6f}")

    # print("\nObjective Function Sensitivity (df*/dp):")
    # print("-" * 40)
    # df_dp = sensitivities['df_dp'].flatten()
    # for j, param in enumerate(param_names):
    #     print(f"  df*/d{param} = {df_dp[j]:+.6f}")

    # # Identify most sensitive parameter
    # max_idx = np.argmax(np.abs(df_dp[:3]))  # Only check p0, p1, p2 (not bound)
    # print(f"\nMost sensitive parameter: {param_names[max_idx]} "
    #       f"(df*/d{param_names[max_idx]} = {df_dp[max_idx]:+.6f})")

    # # Verify sensitivities with finite differences
    # print("\n" + "="*60)
    # print("Verification with Finite Differences")
    # print("="*60)

    # eps = 1e-5
    # for j, param in enumerate(['p0', 'p1', 'p2', 'theta0_ub']):
    #     # Save original value
    #     orig_val = prob.get_val(param)[0] if param != 'theta0_ub' else 6.0

    #     # Perturb parameter
    #     if param == 'theta0_ub':
    #         prob.model._design_vars['theta0']['upper'] = 6.0 + eps
    #     else:
    #         prob.set_val(param, orig_val + eps)

    #     prob.set_val('theta0', 5.0)
    #     prob.set_val('theta1', -5.0)

    #     # Re-optimize
    #     prob.run_driver()
    #     f_plus = prob.get_val('f')[0]

    #     # Restore and perturb in other direction
    #     if param == 'theta0_ub':
    #         prob.model._design_vars['theta0']['upper'] = 6.0 - eps
    #     else:
    #         prob.set_val(param, orig_val - eps)

    #     prob.set_val('theta0', 5.0)
    #     prob.set_val('theta1', -5.0)

    #     prob.run_driver()
    #     f_minus = prob.get_val('f')[0]

    #     # Compute finite difference
    #     fd_sensitivity = (f_plus - f_minus) / (2 * eps)

    #     # Restore original
    #     if param == 'theta0_ub':
    #         prob.model._design_vars['theta0']['upper'] = 6.0
    #     else:
    #         prob.set_val(param, orig_val)

    #     print(f"df*/d{param_names[j]}:")
    #     print(f"  UDE:    {df_dp[j]:+.6f}")
    #     print(f"  FD:     {fd_sensitivity:+.6f}")
    #     print(f"  Error:  {abs(df_dp[j] - fd_sensitivity):.2e}")




In [ ]:
"""
Corrected post-optimality sensitivity analysis using OpenMDAO with matrix-free UDE approach.
"""
import itertools
import numpy as np
import openmdao.api as om
import scipy.sparse as sp
from scipy.sparse.linalg import gmres, LinearOperator


class PostOptimalitySensitivitySolver:
    """
    Matrix-free UDE solver for post-optimality sensitivity analysis.
    Uses Hessian-vector products to avoid forming the full Hessian.
    """
    def __init__(self, prob, design_vars=None, parameters=None, outputs=None,
                 hvp_method='forward', hvp_eps=1e-7, verbose=False, driver_scaling=False):
        """
        Initialize the UDE solver.

        Parameters
        ----------
        hvp_method : str
            Method for Hessian-vector products: 'forward', 'central', or 'forward_basepoint'
        hvp_eps : float
            Finite difference step size for Hessian-vector products
        """
        self.prob = prob
        self.design_vars = self.prob.driver._designvars
        self.constraints = self.prob.driver._cons
        self.outputs = outputs
        self.verbose = verbose
        self.hvp_method = hvp_method
        self.hvp_eps = hvp_eps

        param_dict = {meta['prom_name'] : {'size': meta['size'], 'units': meta['units']}
                      for meta in
                      self.prob.model.get_io_metadata(is_indep_var=True, is_design_var=False).values()}
        if parameters:
            if not set(parameters).issubset(param_dict.keys()):
                missing = set(parameters) - set(param_dict.keys())
                raise KeyError(f"The following specified parameters are not independent variables in the model:\n{missing}")
            self.parameters = {k: param_dict[k] for k in parameters if k in param_dict}
        else:
            self.parameters = param_dict

        # self.n_lambda = len(constraints)
        self.n_p = len(parameters)
        self.n_f = len(outputs)

        # Store the optimal point
        self.x_opt = np.array([prob[var] for var in design_vars])

        # Counter for gradient evaluations
        self.grad_eval_count = 0

        # Store information on the active constraints and design vars
        driver = self.prob.driver
        self._active_dvs, self._active_cons = driver.compute_lagrange_multipliers(driver_scaling=driver_scaling)

        self.n_theta = 0
        idx0 = 0
        active_dv_idxs = []
        for dv, dv_meta in self.prob.driver._designvars.items():
            size = dv_meta['size']
            self.n_theta += size
            if dv in self._active_dvs:
                active_dv_idxs.append(idx0 + self._active_dvs[dv]['indices'])
            idx0 += size
        self._active_dv_idxs = np.concatenate(active_dv_idxs) if active_dv_idxs else None

        self._n_active_cons = 0 if not self._active_cons else sum([act['indices'] for act in self._active_cons.values()])
        self._n_active_dvs = 0 if not self._active_dvs else sum([act['indices'] for act in self._active_dvs.values()])
        self._n_lambda = self._n_active_cons + self._n_active_dvs

        self.total_size = self.n_p + self.n_theta + self._n_lambda + self.n_f

        # The portion of the constraint jacobian due to the active dvs
        self._jac_g_theta = np.eye(self.n_theta)[self._active_dv_idxs, :] if self._active_dv_idxs else None

    def gradient_lagrangian(self, x_perturbed=None):
        """
        Compute gradient of Lagrangian with evaluation counting.
        """
        self.grad_eval_count += 1

        if x_perturbed is not None:
            # Set perturbed values
            for i, var in enumerate(self.design_vars):
                self.prob[var] = x_perturbed[i]
            self.prob.run_model()

        # Rest of implementation...
        grad_f = self.prob.compute_totals(
            of=self.outputs[0],
            wrt=self.design_vars,
            return_format='array'
        ).flatten()

        grad_L = grad_f.copy()

        for i, con in enumerate(self.constraints):
            grad_g = self.prob.compute_totals(
                of=con,
                wrt=self.design_vars,
                return_format='array'
            ).flatten()

            if hasattr(self, 'lambda_vals'):
                grad_L -= self.lambda_vals[i] * grad_g

        return grad_L

    def hessian_vector_product(self, v, eps=1e-7, method='forward'):
        """
        Compute Hessian-vector product ∇²L · v using finite differences.

        Parameters
        ----------
        v : array
            Vector to multiply with Hessian
        eps : float
            Finite difference step size
        method : str
            'forward': H·v ≈ (∇L(x + εv) - ∇L(x)) / ε  (2 gradient evals per call)
            'central': H·v ≈ (∇L(x + εv) - ∇L(x - εv)) / (2ε)  (2 gradient evals per call)
            'forward_basepoint': H·v ≈ (∇L(x + εv) - ∇L(x)) / ε  (1 gradient eval if ∇L(x) cached)

        Returns
        -------
        array
            Hessian-vector product
        """
        # Save current state
        x_current = self.x_opt.copy()

        if method == 'forward':
            # Forward difference - only need x + eps*v
            x_plus = x_current.ravel() + eps * v.ravel()
            grad_L_plus = self.gradient_lagrangian(x_plus)

            # Restore state and compute gradient at base point
            for i, var in enumerate(self.design_vars):
                self.prob[var] = x_current[i]
            self.prob.run_model()
            grad_L_base = self.gradient_lagrangian()

            return (grad_L_plus - grad_L_base) / eps

        elif method == 'central':
            # Central difference - need both x ± eps*v
            x_plus = x_current.ravel() + eps * v.ravel()
            grad_L_plus = self.gradient_lagrangian(x_plus)

            x_minus = x_current.ravel() - eps * v.ravel()
            grad_L_minus = self.gradient_lagrangian(x_minus)

            # Restore original state
            for i, var in enumerate(self.design_vars):
                self.prob[var] = x_current[i]
            self.prob.run_model()

            return (grad_L_plus - grad_L_minus) / (2 * eps)

        elif method == 'forward_basepoint':
            # Forward difference using cached gradient at base point
            # This is most efficient when solving multiple RHS with same A matrix

            # Cache gradient at base point if not already done
            if not hasattr(self, '_cached_grad_L_base'):
                for i, var in enumerate(self.design_vars):
                    self.prob[var] = x_current[i]
                self.prob.run_model()
                self._cached_grad_L_base = self.gradient_lagrangian()

            # Compute gradient at perturbed point
            x_plus = x_current.ravel() + eps * v.ravel()
            grad_L_plus = self.gradient_lagrangian(x_plus)

            # Restore original state
            for i, var in enumerate(self.design_vars):
                self.prob[var] = x_current[i]
            self.prob.run_model()

            return (grad_L_plus - self._cached_grad_L_base) / eps

        else:
            raise ValueError(f"Unknown method: {method}. Use 'forward', 'central', or 'forward_basepoint'")

    def constraint_jacobian(self):
        """
        Get Jacobian of constraints ∇g.
        """
        jac = []
        for con in self.constraints:
            grad_g = self.prob.compute_totals(
                of=con,
                wrt=self.design_vars,
                return_format='array'
            ).flatten()
            jac.append(grad_g)
        return np.array(jac)

    def compute_active_constraint_jacobian(self):
        """
        Get Jacobian of active constraints and bounds ∇g.
        """
        jac_block_rows = []
        for con, act_meta in self._active_cons.items():
            grad_g = self.prob.compute_totals(
                of=con,
                wrt=self.design_vars,
                return_format='array'
            )
            jac_block_rows.append(sp.csr_matrix(grad_g[act_meta['indices'], ...]))

        jac_block_rows.append(self._jac_g_theta)

        return sp.vstack(jac_block_rows)

    def compute_active_constraint_jvp(self, v_theta, mode='fwd'):
        """
        Compute the jacobian vector product for the problem jacobian and
        the associated vector v_theta.
        """
        result = np.zeros((self.n_theta, 1))
        # In forward mode, convert v_theta to seeds.
        dv_seeds = {}
        theta_idx0 = 0
        # jvp_active_dvs = []
        for dv, dv_meta in self.prob.driver._designvars.items():
            dv_size = dv_meta['size']
            if np.any(v_theta[theta_idx0: theta_idx0 + dv_size]):
                dv_seeds[dv] = v_theta[theta_idx0: theta_idx0 + dv_size]
            theta_idx0 += dv_size
            # We need to append the block due to the active design variables
            # to the bottom of the jvp, so do that here
            # if dv in self._active_dvs:
            #     result += v_theta[self._active_dvs['indices']]
        # block_rows = []
        self.prob.model.run_linearize(True)
        row0 = 0
        for con, act_meta in self._active_cons.items():
            jvp = self.prob.compute_jacvec_product(of=[con],
                                                   wrt=list(dv_seeds.keys()),
                                                   seed=dv_seeds,
                                                   mode='fwd')[con]
            n_active = len(act_meta['indices'])
            result[row0:row0 + n_active] += jvp[act_meta['indices']]
            # jvp contains an element for every scalar constraint
            # extract the active ones
            # block_rows.append(jvp[act_meta['indices']])
            row0 += len(act_meta['indices'])

        result[:, 0] += v_theta

        return result

    def ude_matvec(self, vec):
        """
        Matrix-vector product for the UDE system matrix.
        """
        self.matvec_count += 1

        if self.verbose:
            print(f"\nMatvec call #{self.matvec_count}")
            print(f"  Input norm: {np.linalg.norm(vec):.6e}")
            print(f"  Non-zeros: {np.count_nonzero(vec)}")

        # Extract blocks from input vector
        idx = 0
        v_p = vec[idx:idx+self.n_p]
        idx += self.n_p
        v_theta = vec[idx:idx+self.n_theta]
        idx += self.n_theta
        v_lambda = vec[idx:idx+self.n_lambda]
        idx += self.n_lambda
        v_f = vec[idx:idx+self.n_f]

        # Initialize result
        result = np.zeros(self.total_size)
        idx = 0

        # First block row: [I_p, 0, 0, 0] @ v
        result[idx:idx+self.n_p] = v_p
        idx += self.n_p

        # Second block row: [∂∇L/∂p, ∇²L, (∇g)ᵀ, 0] @ v
        row2 = np.zeros(self.n_theta)

        # ∇²L @ v_theta using Hessian-vector product
        if np.any(v_theta):
            Hv = self.hessian_vector_product(v_theta)
            row2 += Hv

        # (∇g)ᵀ @ v_lambda
        if np.any(v_lambda):
            jac_g = self.constraint_jacobian()
            row2 += jac_g.T @ v_lambda

        # ∂∇L/∂p @ v_p (often small near optimum, approximating as zero)
        # Could be computed via finite differences if needed

        result[idx:idx+self.n_theta] = row2
        idx += self.n_theta

        # Third block row: [∂g/∂p, ∇g, 0, 0] @ v
        row3 = np.zeros(self.n_lambda)

        # ∇g @ v_theta
        if np.any(v_theta):
            jac_g = self.compute_active_constraint_jacobian()
            row3 += jac_g.dot(v_theta)

        # ∂g/∂p @ v_p
        if np.any(v_p):
            for i, con in enumerate(self.constraints):
                grad_g_p = self.prob.compute_totals(
                    of=con,
                    wrt=self.parameters,
                    return_format='array'
                ).flatten()
                row3[i] += np.dot(grad_g_p, v_p)

        result[idx:idx+self.n_lambda] = row3
        idx += self.n_lambda

        # Fourth block row: [-∂f/∂p, -∂f/∂θ, 0, I_f] @ v
        row4 = v_f.copy()  # I_f @ v_f

        if np.any(v_p):
            df_dp = self.prob.compute_totals(
                of=self.outputs[0],
                wrt=self.parameters,
                return_format='array'
            ).flatten()
            row4 -= np.dot(df_dp, v_p)

        if np.any(v_theta):
            df_dtheta = self.prob.compute_totals(
                of=self.outputs[0],
                wrt=self.design_vars,
                return_format='array'
            ).flatten()
            row4 -= np.dot(df_dtheta, v_theta)

        # Fourth block row via jvp: [-∂f/∂p, -∂f/∂θ, 0, I_f] @ v
        block_row_4 = v_f.copy()  # I_f @ v_f
        df_dp_accum = itertools.product(list(self.outputs.keys()), list(self.parameters.keys()))


        if np.any(v_p):
            # Break v_p into a seed for each parameter (wrt)
            p_idx0 = 0
            vp_seeds = {}
            for param_name, param_meta in self.parameters.items():
                p_size = param_meta['size']
                if np.any(v_theta[p_idx0: p_idx0 + p_size]):
                    vp_seeds[param_name] = v_p[p_idx0: p_idx0 + p_size]
                p_idx0 += p_size
            jvp_dfdp_v_p = self.prob.compute_jacvec_product(of=self.outputs,
                                                            wrt=list(self.parameters.keys()),
                                                            mode='fwd',
                                                            seed=vp_seeds)


            # df_dp = self.prob.compute_totals(
            #     of=self.outputs[0],
            #     wrt=self.parameters,
            #     return_format='array'
            # ).flatten()
            # row4_2 -= np.dot(df_dp, v_p)

        if np.any(v_theta):
            df_dtheta = self.prob.compute_totals(
                of=self.outputs[0],
                wrt=self.design_vars,
                return_format='array'
            ).flatten()
            row4_2 -= np.dot(df_dtheta, v_theta)

        result[idx:idx+self.n_f] = row4_2

        if self.verbose:
            print(f"  Output norm: {np.linalg.norm(result):.6e}")

        return result

    def solve_sensitivities(self, tol=1e-6, maxiter=1000):
        """
        Solve for post-optimality sensitivities using GMRES.
        """
        # Reset counter
        self.matvec_count = 0

        # Create linear operator
        A_op = LinearOperator(
            (self.total_size, self.total_size),
            matvec=self.ude_matvec
        )

        # Test the operator first
        if self.verbose:
            print("\n" + "="*60)
            print("Testing LinearOperator")
            print("="*60)
            test_vec = np.zeros(self.total_size)
            test_vec[0] = 1.0
            test_result = A_op.matvec(test_vec)
            print(f"Test result norm: {np.linalg.norm(test_result):.6e}")

        # Storage for solutions
        X = np.zeros((self.total_size, self.total_size))

        print(f"\nSolving UDE system ({self.total_size}x{self.total_size})...")
        print(f"Parameters: {self.parameters}")
        print(f"Design vars: {self.design_vars}")
        print(f"Constraints: {self.constraints}")
        print(f"Outputs: {self.outputs}")

        # Solve for each column of the identity matrix
        col = 0

        # Columns for I_p
        for i in range(self.n_p):
            print(f"\n  Solving for parameter {self.parameters[i]}...")
            rhs = np.zeros(self.total_size)
            rhs[i] = 1.0

            # Use a non-zero initial guess can help convergence
            x0 = rhs.copy() * 0.1

            sol, info = gmres(A_op, rhs, x0=x0, atol=tol, rtol=tol, maxiter=maxiter)

            if info == 0:
                pass
                # print(f"    Converged! ||residual|| = {np.linalg.norm(A_op.matvec(sol) - rhs):.6e}")
            else:
                print(f"    Warning: GMRES info={info}")

            X[:, col] = sol
            col += 1

        # Columns for I_theta
        for i in range(self.n_theta):
            print(f"\n  Solving for design var {self.design_vars[i]} sensitivity...")
            rhs = np.zeros(self.total_size)
            rhs[self.n_p + i] = 1.0

            x0 = rhs.copy() * 0.1

            sol, info = gmres(A_op, rhs, x0=x0, atol=tol, rtol=tol, maxiter=maxiter)

            if info == 0:
                pass
                # print(f"    Converged! ||residual|| = {np.linalg.norm(A_op.matvec(sol) - rhs):.6e}")
            else:
                print(f"    Warning: GMRES info={info}")

            X[:, col] = sol
            col += 1

        # Columns for I_lambda
        for i in range(self.n_lambda):
            print(f"\n  Solving for constraint {self.constraints[i]} multiplier sensitivity...")
            rhs = np.zeros(self.total_size)
            rhs[self.n_p + self.n_theta + i] = 1.0

            x0 = rhs.copy() * 0.1

            sol, info = gmres(A_op, rhs, x0=x0, atol=tol, rtol=tol, maxiter=maxiter)

            if info == 0:
                pass
                # print(f"    Converged! ||residual|| = {np.linalg.norm(A_op.matvec(sol) - rhs):.6e}")
            else:
                print(f"    Warning: GMRES info={info}")

            X[:, col] = sol
            col += 1

        # Columns for I_f
        for i in range(self.n_f):
            print(f"\n  Solving for output {self.outputs[i]} sensitivity...")
            rhs = np.zeros(self.total_size)
            rhs[self.n_p + self.n_theta + self.n_lambda + i] = 1.0

            x0 = rhs.copy() * 0.1

            sol, info = gmres(A_op, rhs, x0=x0, atol=tol, rtol=tol, maxiter=maxiter)

            if info == 0:
                pass
                # print(f"    Converged! ||residual|| = {np.linalg.norm(A_op.matvec(sol) - rhs):.6e}")
            else:
                print(f"    Warning: GMRES info={info}")

            X[:, col] = sol
            col += 1

        print(f"\nTotal matvec calls: {self.matvec_count}")

        # Extract sensitivities
        idx_theta = self.n_p
        idx_lambda = self.n_p + self.n_theta
        idx_f = self.n_p + self.n_theta + self.n_lambda

        sensitivities = {
            'dtheta_dp': X[idx_theta:idx_lambda, :self.n_p],
            'dlambda_dp': X[idx_lambda:idx_f, :self.n_p],
            'df_dp': X[idx_f:, :self.n_p]
        }

        return sensitivities


In [82]:
main()


Running Optimization

Optimization Results
θ₀* = 6.000000
θ₁* = -6.000000
f* = -26.000000
g₁ = 0.000000e+00 (should be ~0)
g₂ = -0.000000
Bound active: bound_active

Computing Lagrange Multipliers
∇f = [-8.8817842e-16  2.0000000e+00]
∇g1 = [1. 1.]
∇g2 = [ 1. -0.]
Lagrange multipliers: [-2.  2.]
KKT residual: [8.8817842e-16 4.4408921e-16] (should be ~0)

Computing Post-Optimality Sensitivities
[[1. 1.]]
[[2.]
 [0.]]
